5. Работа с векторами слов

стр 100

Наборы вещественных чисел, отражающие смысл слов естественного языка, называются векторами слов:

В этой главе векторы слов будут использоваться для вычисления семантического подобия различных текстов, что позволит категоризировать тексты в соответствии с их тематикой.

Пространство векторов слов можно представить в виде облака, где векторы располагаются тем ближе, чем ближе значения

#### Метод similarity

similarity, который используется для вычисления оценки семантического подобия двух объектов-контейнеров произвольного типа посредством сравнения их векторов слов.

In [4]:
import spacy as sp
nlp = sp.load('ru_core_news_lg')

In [5]:
doc = nlp("Я хочу на море")
clice = doc[2:5] # на море

In [6]:
doc.similarity(clice) # схожесть от 0 до 1. 0,88 очень высоко

0.9103373697881717

In [7]:
doc2=nlp('Я хочу на мандарин')

In [8]:
doc.similarity(doc2)

0.720314515118873

// можно сравнивать почти что угодно. doc и span// token и span

c.м стр 105


Метод similarity() распознает слова, которые относятся к одной или нескольким близким категориям и часто встречаются во взаимосвязанных контекстах, а также возвращает для подобных слов высокие значения уровня подобия.

Метод similarity берет на себя вычисление семантического подобия, но для извлечения пользы из этих расчетов необходимо подобрать правильные ключевые слова для сравнения.

In [9]:
doc.noun_chunks

##### Выбор ключевых слов для вычисления семантического подобия

Например если нужно искать в тексте главную мысль о том, где растут яблоки; то нужно будет сравнивать слово географичесокй категории(и/или фруктовой) с кусками исходного текста и искать большую близость 

#### Установка пакетов векторов слов

Если в среде Python установлена модель spaCy, векторы слов можно использовать сразу. Но не забывайте, что есть возможность установить сторонний пакет векторов слов. Разные статистические модели используют различные векторы слов, поэтому результаты операций могут незначительно отличаться в зависимости от используемой модели. Испытайте несколько моделей, чтобы выяснить, какая из них лучше подходит для вашего конкретного приложения.

#### Пользуемся векторами слов, прилагаемыми к моделям spaCy

#### Применение семантического подобия для задач категоризации

допустим есть три предложения. Необходимо выбрать из них, то, которое связано с фруктами.

In [20]:
text = "Хочу жить в доме на дереве. n\
Мне не нравтся ваша идея,статья полный бред. n\
Хочу скушать апельсин и посадить его в своем саду."

1 и самый простой способо - сравнить слово "фрукты" с каждым предложением и выбрать наиболльшее подоюие 

In [21]:
doc = nlp(text)
marker = nlp("фрукты")[0]
for sent in doc.sents: # идём по предложениям
    print(sent.text)
    print('similarity to', marker.text, 'is', marker.similarity(sent),'\n')

Хочу жить в доме на дереве.
similarity to фрукты is 0.15878769755363464 

nМне не нравтся ваша идея,статья полный бред.
similarity to фрукты is 0.05543072894215584 

nХочу скушать апельсин и посадить его в своем саду.
similarity to фрукты is 0.3621806800365448 



Видно, что если выбрать максимльную similitart, то результат будет верным

Но видно, что сценарий показывает, что сходство есть и с первым предложением

Но обратите внимание: сценарий также определил, что с фруктами в какой-то степени связано и третье предложение. Вероятно, это произошло из-за слова tree — ведь многие фрукты растут на деревьях. Наивно полагать, что алгоритм вычисления подобия «знает», что апельсин и цитрус — фрукты. Ему известно только то, что соответствующие слова (orange и citrus) часто встречаются в одном контексте со словом fruit, а значит, они должны быть близки в пространстве векторов слов. Но слово tree также нередко встречается в одном контексте со словом fruit. Например, в текстах вполне можно встретить фразу fruit tree. Поэтому степень подобия слова fruits (и его леммы — слова fruit) и слова tree близка к результату, полученному для слов citrus и fruits.

если текст большой, его будет сложно сравнивать для категоризации. Поэтому ему требуется предварительная обработка. Выделить из текста значимые слова.

#### Выделение существительных как шаг предварительной обработки

не всегда нужно выделять существиетльнве. иногда полезно выделять и глаголы.

In [22]:
text = "Хочу жить в доме на дереве. n\
Мне не нравтся ваша идея,статья полный бред. n\
Хочу скушать апельсин и посадить его в своем саду."

In [30]:
doc = nlp(text)
marker = nlp("фрукты")[0]
similarity = {}
for i, sent in enumerate(doc.sents):
     noun_span_list = [sent[j].text for j in range(len(sent)) if sent[j].pos_ == 'NOUN']
     noun_span_str = ' '.join(noun_span_list)
     print(i,noun_span_str)
     noun_span_doc = nlp(noun_span_str)
     similarity.update({i:marker.similarity(noun_span_doc)})


print(similarity)



0 доме дереве
1 идея статья бред
2 апельсин саду
{0: 0.16821357624654942, 1: -0.03267072353415493, 2: 0.4680054051381411}


Видно, что фрукты тепрь предложения с фруктами теперь определяются лучше. 

попробуем сравнивать каждое существительное в предложение и определять, какое из них больше приближено к marker - это значение и возьмём за оценку близости предложения к результату

In [38]:
doc = nlp(text)
marker = nlp("фрукты")[0]
similarity = {}
out_chunk = {} # слова наиболее полезные для определения категории текстав в целом
for i, sent in enumerate(doc.sents):
     rate = []
     noun_span_list = [sent[j].text for j in range(len(sent)) if sent[j].pos_ == 'NOUN']
     print(i,noun_span_list)
     for j, chunk in enumerate(noun_span_list):
        item_doc = nlp(chunk)[0]
        rate.append(marker.similarity(item_doc))

     print(rate)
     similarity.update({i:max(rate)})
     # + запишем словов наиболее полезное в словарь - out_chunk


print(similarity)


0 ['доме', 'дереве']
[0.10583904385566711, 0.1570703387260437]
1 ['идея', 'статья', 'бред']
[0.011179545894265175, -0.045194111764431, -0.030149759724736214]
2 ['апельсин', 'саду']
[0.37877795100212097, 0.32962870597839355]
{0: 0.1570703387260437, 1: 0.011179545894265175, 2: 0.37877795100212097}


#### Выделение и сравнение именованных сущностей

В некоторых случаях вместо выделения всех существительных из сравниваемых текстов имеет смысл выделять лишь определенный вид, например именованные сущности.

In [53]:
doc1 = nlp("Гугл это компания по поиску инормации в интернете. n\
     Пользователи часто пользуются Гугл, чтобы найти что-нибудь ")

doc2 = nlp("Винда это симейство операционных систем созданное майрософт. n\
    Компания также производить много софта для компьютеров. ")

doc3 = nlp("Эльбрус это самая высока гора в европе. Многие путешествиники мечтают побывать n\
    на его склонах. ")

# попробуем определить, какой текст связан с технологиями. ДЛя этого используем именованные
# сущности   

In [54]:
docs = [doc1,doc2,doc3]

spans = {}
for j, doc in enumerate(docs):
    named_entity_span = [doc[i].text for i in range(len(doc)) if doc[i].ent_type != 0]
    print(named_entity_span)
    named_entity_span = ' '.join(named_entity_span)
    named_entity_span = nlp(named_entity_span)
    spans.update({j:named_entity_span})


['Гугл']
[]
['европе']


In [55]:
print('doc1 is similar to doc2:',spans[0].similarity(spans[1]))
print('doc1 is similar to doc3:',spans[0].similarity(spans[2]))
print('doc2 is similar to doc3:',spans[1].similarity(spans[2]))

doc1 is similar to doc2: 0.0
doc1 is similar to doc3: 0.0
doc2 is similar to doc3: 0.0


<ipython-input-55-3ba975a8ccfc>:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print('doc1 is similar to doc2:',spans[0].similarity(spans[1]))
<ipython-input-55-3ba975a8ccfc>:2: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print('doc1 is similar to doc3:',spans[0].similarity(spans[2]))
<ipython-input-55-3ba975a8ccfc>:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print('doc2 is similar to doc3:',spans[1].similarity(spans[2]))


#### Резюме
В этой главе вам довелось поработать с векторами слов — вещественными векторами, отражающими значение слов. Благодаря подобному представлению можно математически определять семантическое подобие языковых единиц, что очень важно при категоризации текстов.
Но для успешной работы этого математического подхода необходима предварительная обработка текстов: с ее помощью можно свести текст до наиболее важных для его смысла слов. Если текст очень большой, стоит выбрать из него именованные сущности, поскольку они лучше всего описывают категорию, к которой этот текст следует отнести.